# Algorithme des *k* plus proches voisins

En guise d'initiation au problème d'apprentissage supervisé, nous allons implanter et manipuler l'algorithme des *k* plus proches voisins.

L'objectif de ce TP est de faire reconnaître automatiquement par l'ordinateur des chiffres manuscrits (pour lire des chèques, lire des codes postaux par exemple). ![enveloppe](lettres.jpg)

## 1. Lecture du jeux d'exemples

Avant même d'implanter l'algorithme des *k* plus proches voisins, nous avons besoin d'exemples qui seront traités par l'algorithme. Aussi, commençons par lire un jeu de données.

Il s'agit d'un jeu de données très célèbre appelé MNIST. Il est constitué d'un ensemble de 70000 images, 28x28 pixels, en noir et blanc annoté du chiffre correspondant (entre 0 et 9). Ce jeu utilise des données réelles qui ont déjà été pré-traitées pour être plus facilement utilisables par un algorithme.

![Un extrait du type d'images que l'on trouve dans le dataset MNIST](extrait_MNIST.png)

Le jeu de données est relativement petit mais pour l'algorithme des *k* plus proches voisins, il est déjà trop gros pour obtenir rapidement des résultats. On va donc effectuer un échantillonnage et travailler sur seulement 5000 données.

In [ ]:
from sklearn.datasets import fetch_openml
import random

#Récupère le jeu de données MNIST
mnist = fetch_openml('mnist_784',cache=True, version = 1, data_home = '/home/pascal/Dropbox/cours/NSI/projet_bloc1/')

In [43]:
#Créé un échantillon de 5000 données à partir du jeu de données MNIST
tailleEchantillon = 5000
echantillon = sorted(random.sample(range(70000), tailleEchantillon))

listeImages = mnist.data[echantillon]
listeChiffres = mnist.target[echantillon]

#Récupère une image qui n'est pas dans notre échantillon
test = random.randint(1,70000)-1
while test in sample:
    test = random.randint(1,70000)-1

chiffreTest = mnist.target[test]
imageTest = mnist.data[test]

On obtient deux listes, **listeImages** et **listeChiffres**.
<ul>
<li> listeImages contient les images sous forme d'une liste 28*28 = 784 entiers compris entre 0 et 255 correspondant aux différentes nuances de gris (255 étant blanc et 0 noir)</li>
<li> listeChiffres contient les chiffres correspondant à l'image</li>
</ul>

Afficher la liste correspondant à la 2423ème image du jeu de données. A quel chiffre corrrespond cette image ?

In [53]:
print(listeImages[2423],'\n')

print('Le chiffre représenté est :', listeChiffres[2423])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  64. 128. 191. 255.
 255. 255. 128.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. 128. 255. 255. 255. 255.
 255. 255. 255. 128.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.  64. 255. 255. 255. 255. 128. 128.
 191. 255. 255. 128.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

## 2. Déterminer le plus proche voisin d'un point
### Distance entre deux images
Avant toute chose, il nous faut définir une fonction qui mesure la distance entre deux images. Les images sont des images [PGM](https://fr.wikipedia.org/wiki/Portable_pixmap#Fichier_binaire_2), c'est à dire en niveau de gris.

* Pour tous les pixels de l'image on calcule la différence des valeurs d'intensité, dont on supprime le signe (abs)
* On somme toutes ces différences

![comparaison](distance_eleve.png)

1. Pour les deux images précédentes, déterminez la distance.
2. Écrire une fonction qui calcule la distance entre deux images. Une image sera représentée comme une liste des valeurs de gris des pixels

In [31]:
def distance (image1, image2):
    somme = 0
    for i in range(784):
        somme += abs(image1[i] - image2[i])
    return (somme)

In [45]:
image1 = listeImages[2423]
distance(image1,imageTest)

35578.0

### Le plus proche voisin
Écrire une fonction lePlusProcheVoisin qui prend en paramètre une image (une liste de 784 entiers compris entre 0 et 255) et renvoie l'indice dans data du plus proche voisin.

In [46]:
def lePlusProcheVoisin (image):
    lePlusPres = 0
    distanceMin = float("inf")
    for i in range(len(listeImages)):
        di = distance (image, listeImages[i])
        if di < distanceMin:
            lePlusPres = i
            distanceMin = di
    return (lePlusPres)

In [60]:
indice = lePlusProcheVoisin(imageTest)
print(listeChiffres[indice])
print(chiffreTest)

0
0


## 3. Déterminer les *k* plus proches voisins d'un point
Écrire une fonction lesKplusProchesVoisins qui prend en paramètre une image et une valeur de k et renvoie la liste des indices dans data des k plus proches voisins.
Quand vous prenez k = 1, cette fonction doit renvoyer le même résultat que la précédente, mis à part le fait que lePlusProcheVoisin renvoie une valeur numérique alors que lesKplusProchesVoisins renvoie une liste d'un élément.

In [58]:
#Première version qui boucle k fois sur listeImages
def lesKplusProchesVoisins (image, k):
    listeDesDistances = []
    for img in listeImages:
        listeDesDistances.append (distance (image, img))
    voisins = []
    for i in range (k):
        p = float ("inf")
        for j in range (len (data)):
            if listeDesDistances [j] < p and j not in voisins:
                p = listeDesDistances [j]
                indice = j
        voisins. append (indice)
    return (voisins)


#Deuxième version de l'algorithme qui utilise une fonction de trie
from operator import itemgetter

def lesKplusProchesVoisins (image, k):
    listeDesDistances = []
    for i in range(len(listeImages)):
        di = distance(image, listeImages[i])
        listeDesDistances.append((i,di))
    listeDesDistances.sort(key=itemgetter(1))
    voisins=[]
    for i in range(k):
        voisins.append(listeDesDistances[i][0])
    return (voisins)

In [61]:
voisins = lesKplusProchesVoisins (imageTest, 10)
print([listeChiffres[indice] for indice in voisins])

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


## 4. Prédire l'étiquette d'une donnée

Écrire une fonction *predire* qui prend en paramètre une image dans le même format que celles de data et un entier *k* et retourne le chiffre qui est prédit, c'est à dire le chiffre qui est supposé être représenté sur l'iamge.
On décide du chiffre représenté sur l'image en appliquant un choix à la majorité, à savoir le chiffre qui apparaît majoritairement sur les *k* plus proches voisins.

In [67]:
def predire (image,k):
    voisins = lesKplusProchesVoisins(image,k)
    decomptes = {'0':0, '1':0, '2':0 , '3':0, '4':0, '5':0, '6':0, '7':0 , '8':0, '9':0}
    for indice in voisins:
        decomptes[listeChiffres[indice]] += 1
    plusGrandDecompte = 0
    for i in '0123456789':
        if decomptes [i] > plusGrandDecompte:
                plusGrandDecompte = decomptes [i]
                indice = i
    return indice


In [68]:
print(predire(imageTest,5))
print(chiffreTest)

0
0


## 5. Optimisation

In [86]:
#séparation du jeu de données en training set et testing set (0.8/0.2)
tailleEchantillon = 5000
echantillon = random.sample(range(70000), tailleEchantillon)

echantillonEntrainement = echantillon[:4000]
echantillonTest = echantillon[4000:]

imagesEntrainement = mnist.data[echantillonEntrainement]
chiffresEntrainement = mnist.target[echantillonEntrainement]

imagesTest = mnist.data[echantillonTest]
chiffresTest = mnist.target[echantillonTest]

#redéfinition de la fonction lesKplusProchesVoisins pour lui donner en paramètre l'ensemble d'entrainement
def lesKplusProchesVoisins (image, k,entrainementData):
    listeDesDistances = []
    for i in range(len(entrainementData)):
        di = distance(image, entrainementData[i])
        listeDesDistances.append((i,di))
    listeDesDistances.sort(key=itemgetter(1))
    voisins=[]
    for i in range(k):
        voisins.append(listeDesDistances[i][0])
    return (voisins)
#redéfinition de la fonction prédire pour lui donner les images d'entrainement et les chiffres correspondant en paramètre
def predire (image,k,entrainementTarget,entrainementData):
    voisins = lesKplusProchesVoisins(image,k,entrainementData)
    decomptes = {'0':0, '1':0, '2':0 , '3':0, '4':0, '5':0, '6':0, '7':0 , '8':0, '9':0}
    for indice in voisins:
        decomptes[entrainementTarget[indice]] += 1
    plusGrandDecompte = 0
    for i in '0123456789':
        if decomptes [i] > plusGrandDecompte:
                plusGrandDecompte = decomptes [i]
                indice = i
    return indice

def tauxErreur(k):
    '''
    calcule le taux d'erreur avec la valeur k pour les k plus proches voisins
    '''
    t=0
    n=len(imagesTest)
    for i in range(n):
        if predire(imagesTest[i],k,chiffresEntrainement,imagesEntrainement)!=chiffresTest[i]:
            t+=1
        if(i % 50 == 0 and i != 0):
            print("Taux d'erreurs pour %s images : %s" % (i,t/i))
    return t/n

def optimisation(n):
    '''
    détermine quelle valeur de k donne la meilleure prédiction, avec k entre 1 et n
    '''
    listeTaux=[tauxErreur(k) for k in range(1,n+1)]
    return listeTaux.index(min(liste_taux))+1

In [87]:
tauxErreur(1)

Taux d'erreurs pour 50 images : 0.04
Taux d'erreurs pour 100 images : 0.03
Taux d'erreurs pour 150 images : 0.06666666666666667
Taux d'erreurs pour 200 images : 0.06
Taux d'erreurs pour 250 images : 0.06
Taux d'erreurs pour 300 images : 0.06
Taux d'erreurs pour 350 images : 0.054285714285714284
Taux d'erreurs pour 400 images : 0.0575
Taux d'erreurs pour 450 images : 0.057777777777777775
Taux d'erreurs pour 500 images : 0.064
Taux d'erreurs pour 550 images : 0.07090909090909091
Taux d'erreurs pour 600 images : 0.07333333333333333
Taux d'erreurs pour 650 images : 0.06923076923076923
Taux d'erreurs pour 700 images : 0.07
Taux d'erreurs pour 750 images : 0.06933333333333333
Taux d'erreurs pour 800 images : 0.07125
Taux d'erreurs pour 850 images : 0.07176470588235294
Taux d'erreurs pour 900 images : 0.07222222222222222
Taux d'erreurs pour 950 images : 0.07473684210526316


0.076